# Linear NN for images

## Imports 

In [1]:
import tensorflow as tf
import numpy as np 
import matplotlib



## Constants

In [2]:
IMG_HEIGTH = 224
IMG_WIDTH = 224
IMG_CHANNELS  = 3


CLASS_NAMES = ['display', 'dandelion', 'roses', 'sunflowers', 'tulips']
